# Used Car Data
## Introduction

The purpose of this script is to clean the data. In doing so, all nulls will be handled and features will be imputed where possible. <br> 

The approach will be to split in the data into test and train intially. Allowing for the data to be imputed using KNN. Once imputed the target variables will be extracted <br> 

The inital raw data is too large for the purpose of the project as it enables time/performance constraints. For the purpose of the project, a sample size will be used. This will be configurable and easily altered to enable the full data set if required. 

In [1]:
import pandas as pd # dataframes 
import numpy as np 
import statistics as stat 
import sys # change the path for folder structure 
sys.path.append(".")
from sklearn.model_selection import train_test_split
import seaborn as sns
sns.set_style('whitegrid')
import matplotlib

## Load the data & check the data types

In [2]:
%%time
#load data into dataframe for cleaning 
raw_data = pd.read_csv("../data/vehicles.csv")
  

CPU times: user 13.3 s, sys: 3.63 s, total: 17 s
Wall time: 17.3 s


In [3]:
raw_data.shape # checking the expected data has arrived

(426880, 26)

## Taking Sample of Total 
Take a sample of the 426880 records as for the project purpose, it needs to be greater than 10k. For interations over cleaning, the performace is time consuming. <br> 

If required to use to the full data set, please skip the sampling cell and edit dropped columns and test and train cells. 

In [4]:
sample = raw_data.sample(100000, replace=True, weights=None, random_state=0)

In [5]:
pd.set_option('display.max_columns', 20 )
sample.head(10)

,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,...,size,type,paint_color,image_url,description,county,state,lat,long,posting_date
305711,7312403979,https://oklahomacity.craigslist.org/ctd/d/okla...,oklahoma city,https://oklahomacity.craigslist.org,0,2001.0,pontiac,montana,NaN,NaN,...,mid-size,mini-van,silver,https://images.craigslist.org/01111_fY2vecoS7R...,2001 PONTIAC MONTANA Offered by: Cars-SUV'...,NaN,ok,35.414450,-97.491710,2021-04-25T12:32:59-0500
117952,7314812555,https://tampa.craigslist.org/hil/ctd/d/tampa-2...,tampa bay area,https://tampa.craigslist.org,19988,2015.0,bmw,428i,NaN,NaN,...,compact,coupe,blue,https://images.craigslist.org/00k0k_2ASDKsugnV...,2015 BMW 4 Series 428i 2dr Coupe SULEV Off...,NaN,fl,28.015670,-82.459780,2021-04-30T11:12:05-0400
152315,7310949462,https://indianapolis.craigslist.org/ctd/d/beec...,indianapolis,https://indianapolis.craigslist.org,3900,2009.0,toyota,yaris,excellent,4 cylinders,...,full-size,hatchback,black,https://images.craigslist.org/01616_16pkt47nM3...,2009 Toyota Yaris with well cared for 188K mil...,NaN,in,39.721344,-86.091048,2021-04-22T14:01:03-0400
358083,7306635391,https://memphis.craigslist.org/ctd/d/desha-196...,memphis,https://memphis.craigslist.org,2200,1964.0,chevrolet,belair,NaN,NaN,...,NaN,NaN,NaN,https://images.craigslist.org/00p0p_9mdWGXH5mi...,1964 belair I don’t see any rust on the outsid...,NaN,tn,35.749899,-91.743845,2021-04-14T09:11:03-0500
359783,7314289506,https://nashville.craigslist.org/ctd/d/murfree...,nashville,https://nashville.craigslist.org,4999,2003.0,toyota,corolla,excellent,4 cylinders,...,NaN,sedan,NaN,https://images.craigslist.org/00K0K_8R41wVDtXI...,2003 Toyota Corolla LE Sedan FOR FAST TEXT RES...,NaN,tn,35.839799,-86.386926,2021-04-29T09:15:52-0500
304137,7305693765,https://lawton.craigslist.org/ctd/d/lawton-201...,lawton,https://lawton.craigslist.org,18590,2018.0,hyundai,sonata se sedan 4d,good,NaN,...,NaN,sedan,white,https://images.craigslist.org/00y0y_86Glvp84gh...,Carvana is the safer way to buy a car During t...,NaN,ok,34.600000,-98.420000,2021-04-12T11:51:22-0500
122579,7315704460,https://atlanta.craigslist.org/sat/ctd/d/jones...,atlanta,https://atlanta.craigslist.org,15995,2012.0,ford,super duty f-250 srw,NaN,8 cylinders,...,NaN,pickup,white,https://images.craigslist.org/00Q0Q_5WTWjmsFKP...,2012 *Ford* *Super Duty F-250 SRW* 4WD Crew Ca...,NaN,ga,33.552199,-84.352522,2021-05-02T09:10:24-0400
86293,7314923295,https://newhaven.craigslist.org/ctd/d/north-br...,new haven,https://newhaven.craigslist.org,3900,2007.0,volvo,s40 2.4i,NaN,5 cylinders,...,NaN,sedan,black,https://images.craigslist.org/00O0O_icBEHOeZ85...,*** 2007 VOLVO S40 4D SEDAN 2.4i Clean ...,NaN,ct,41.332300,-72.780900,2021-04-30T14:13:03-0400
374564,7316659920,https://houston.craigslist.org/ctd/d/houston-2...,houston,https://houston.craigslist.org,13993,2013.0,nissan,pathfinder,NaN,6 cylinders,...,full-size,SUV,red,https://images.craigslist.org/00Q0Q_7vge0rihNY...,Most common questions about this vehicle: Wan...,NaN,tx,29.890360,-95.392527,2021-05-04T08:03:30-0500
211543,7315536299,https://duluth.craigslist.org/ctd/d/minneapoli...,duluth / superior,https://duluth.craigslist.org,22995,2017.0,chevrolet,silverado 1500 ls 4x4,excellent,8 cylinders,...,full-size,pickup,black,https://images.craigslist.org/00202_lqASUsF3rd...,This is the Iconic 2017 Chevrolet Silverado 15...,NaN,mn,45.064500,-93.341300,2021-05-01T17:10:56-0500


The data contains 26 features. <br>
<hr>
ID - Random ID number <br>
url - this is the craigs list url for the ad <br>
region - this is the region within the state of the USA <br>
region_url - The direct url for craigs list of that region <br>
price - The price of the vehcile on sale <br>
year - year of the vehcile on file. <br>
manufacturer - the manufacturer of the vehcile <br>
model - the model of the vehcile, this is free hand entry with no consistency <br>
condition - the condition of the vehcile <br>
cylinders - the engine size <br>
fuel - The type of fuel the vehcile requires <br>
odometer - the number of miles travelled <br> 
title_status - the status of the sale. (clean, rebuilt, lien, salvage, parts only, missing) <br> 
transmission - if the vehcile is auto or manual. (contains other) <br>
VIN - The vehcile identifier number <br> 
drive - they type of drive the vehcile is. (4wd,fwd,rwd) <dr> 
size - a description of the size of the vehcile. ( sub-compact, compact, mid-size, full-size) <br>
type - the type of vehcile it is. (sedan, SUV , pickup, truck, other, coupe, hatchback, wagon, van, convertible, mini-van, bus, offroad) <br>
paint color - the color of the vehcile <br> 
image url - a link to photos of the vehcile <br>
description - personal description of the vehcile from the seller <br>
county - the county of the sale is located <br>
state - the state where the sale is located <br>
lat - latatude of the location <br>
long - longatude of the location <br>
posting data - the date the ad was posted online 





In [ ]:
%%time
sample.info() # get the information breakdown of the data

## Dropping columns no required 

In [6]:
#drop all columns not required for modelling 
sample =sample.drop(columns=['id', 'url','region_url','VIN', 'county', 'lat','long', 'posting_date', 'image_url', 'description', 'model'])

In [ ]:
sample.isnull().sum()

## Calulate the number of NaN's per columns

Creation of a dataframe that stores the caluated nulls for each of the features and % of them are nulls. 

In [9]:
#count up the number of nullss

null_review = sample.isnull().sum()
null_review = pd.DataFrame(null_review, columns=["Null"])
total = len(sample)
null_review['%'] = pd.DataFrame(null_review['Null']/total)
null_review.sort_values('%',ascending=False)


,Null,%
size,71830,0.71830
cylinders,41910,0.41910
condition,40726,0.40726
paint_color,30884,0.30884
drive,30713,0.30713
type,21912,0.21912
manufacturer,4060,0.04060
title_status,1911,0.01911
odometer,981,0.00981
fuel,703,0.00703


## Initial Notes
1. The data shows a number of features that contain null values.
2. There are a number of features that are not required for modelling

## Removing small levels of rows that contain NaN's 

removing the data rows that contails very small numbers of NaNs. Anything less that 5% will remove the rows. The null data frame is recreated to confirm the data has been removed

Only six features contians NaNs

In [10]:

sample = sample.dropna(subset=['year','transmission', 'fuel', 'odometer','title_status', 'manufacturer'])



# Check the value counts for all the categorical features for training data

We are checking the feature for their value types and counts. (NaNs do not appear in this section)

In [ ]:
sample['region'].value_counts()

In [ ]:
# check the catigorical type of data
sample['size'].value_counts()

In [12]:
sample['condition'].value_counts()/len(sample)

good         0.293837
excellent    0.229681
like new     0.051731
fair         0.015340
new          0.002764
salvage      0.001306
Name: condition, dtype: float64

In [ ]:
sample['drive'].value_counts()

In [ ]:
sample['paint_color'].value_counts()

In [ ]:
sample['type'].value_counts()

In [ ]:
sample['manufacturer'].value_counts()

In [ ]:
sample['title_status'].value_counts()

In [ ]:
sample['region'].value_counts()

In [ ]:
sample['fuel'].value_counts()

In [ ]:
sample['transmission'].value_counts()

In [ ]:
sample['state'].value_counts()

## Notes 
1. fuel has records  classed as other. Unable to know or predict the fuel type so these row will be removed
2. Transmission as  records classed as other. Unable to know or predict the transamission type so these rows will be removed. 
3. Type has records classed as other. 
4. parts only from title status will be removed as this will not enable the condition of a vehcile 
5. missing from title status will be removed as this will not enable the condition of a vechile 

## Removing small value types of 'other'

In [13]:
#removing the rows of data as per the notes above 
sample = sample.drop(sample[sample['fuel']=='other'].index | sample[sample['transmission']=='other'].index | sample[sample['type']=='other'].index)


<ipython-input-13-59a6d29e37ef>:2: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead
  sample = sample.drop(sample[sample['fuel']=='other'].index | sample[sample['transmission']=='other'].index | sample[sample['type']=='other'].index)


## Remove Parts only and missing from title status  

In [14]:
#removing the rows of data as per the notes above 
sample = sample.drop(sample[sample['title_status']=='parts only'].index | sample[sample['title_status']=='missing'].index)

<ipython-input-14-24ed1bedb177>:2: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead
  sample = sample.drop(sample[sample['title_status']=='parts only'].index | sample[sample['title_status']=='missing'].index)


# Encoding features
Preparing each of the features for imputation using KNN.

The following features will not need to be encoded as they are numeric already

1. Year
2. price
3. Odometer


In [15]:
# Using the columns names from the train data for encoding both test and train.
cols =list(sample.columns)
remove = ['year','price', 'odometer']
for item in remove:
    cols.remove(item)
cols

['region',
 'manufacturer',
 'condition',
 'cylinders',
 'fuel',
 'title_status',
 'transmission',
 'drive',
 'size',
 'type',
 'paint_color',
 'state']

In [16]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder

In [17]:
sample

,region,price,year,manufacturer,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,state
117952,tampa bay area,19988,2015.0,bmw,NaN,NaN,gas,105716.0,clean,automatic,NaN,compact,coupe,blue,fl
152315,indianapolis,3900,2009.0,toyota,excellent,4 cylinders,gas,188697.0,clean,automatic,fwd,full-size,hatchback,black,in
359783,nashville,4999,2003.0,toyota,excellent,4 cylinders,gas,146000.0,clean,automatic,fwd,NaN,sedan,NaN,tn
122579,atlanta,15995,2012.0,ford,NaN,8 cylinders,diesel,339236.0,clean,automatic,4wd,NaN,pickup,white,ga
86293,new haven,3900,2007.0,volvo,NaN,5 cylinders,gas,154273.0,clean,automatic,4wd,NaN,sedan,black,ct
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138819,twin falls,44999,2019.0,ford,NaN,NaN,gas,24363.0,clean,automatic,4wd,NaN,truck,white,id
285771,new hampshire,20995,2016.0,ram,excellent,NaN,gas,117175.0,clean,automatic,4wd,NaN,truck,NaN,nh
382679,san antonio,3995,2004.0,gmc,excellent,8 cylinders,gas,135000.0,clean,automatic,4wd,full-size,SUV,red,tx
356850,memphis,6491,2011.0,nissan,NaN,4 cylinders,gas,164049.0,clean,automatic,fwd,mid-size,sedan,white,tn


## Encode sample data 


In [18]:
%%time
#created a dict to hold the encoders for each column
encoder_dict = {}

#loop through the dataframe and extract non-null, encode them and place them back. 
for name in cols:
    #encoder for the column
    encoder_dict[name] = OrdinalEncoder()
    
    #select no nulls 
    col = sample[name]
    col_notnull = col[col.notnull()]
    reshape = col_notnull.values.reshape(-1,1)

    #encode non-nills 
    encoded_vals = encoder_dict[name].fit_transform(reshape)
    sample.loc[col.notnull(), name] = np.squeeze(encoded_vals)

CPU times: user 230 ms, sys: 20.8 ms, total: 251 ms
Wall time: 250 ms


# Impute NaNs using KNN 

# Impute Data

In [19]:
%%time
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=5)
imputer.fit(sample)


CPU times: user 90.9 ms, sys: 32.2 ms, total: 123 ms
Wall time: 143 ms


KNNImputer()

In [20]:
%%time
impute_sample = imputer.transform(sample)
sample_df = pd.DataFrame(data = impute_sample, columns= sample.columns)

CPU times: user 7min 7s, sys: 1min 40s, total: 8min 47s
Wall time: 6min 24s


In [21]:
sample_df.isnull().any().sum()


0

## Split into dependant and target datasets 


In [22]:
X = sample_df
y= sample_df['condition']
del X['condition']

In [23]:
X_train , X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=12345 )

## Can be used to undo the encoding 

In [ ]:
# Loop the data and reverse the encoding to create the orginal but imputed dataframe. 
for col in cols:
    #encoder for the columns
    reshape_col = df[col].values.reshape(-1,1)
    df[col] = encoder_dict[col].inverse_transform(reshape_col)

In [ ]:
%store X_train
%store y_train
%store X_test
%store y_test
